# Building a Zarr Storage for EO Data

Necessary imports

In [1]:
import zarr
import numpy as np
import pystac_client as pc
import xarray as xr
import rioxarray
import pandas as pd
from datetime import datetime, timezone
from tqdm import tqdm
import BuildZarrStore as bzs
import stackstac
from pyproj import CRS
from dask.distributed import LocalCluster
#os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

In [2]:
client = LocalCluster().get_client()
client.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

### Get item collection from STAC

In [4]:
from pystac import Item
import requests

def get_tif_size(item: Item, band="VV"):
    url = item.assets[band].href
    try:
        r = requests.head(url, allow_redirects=True, timeout=10)
        size = int(r.headers["Content-Length"])
        return size / (1024 ** 2)  # Size in MB
    except Exception as e:
        print(f"Error for {url}: {e}")
        return None

In [5]:
pc_client = pc.Client.open("https://stac.eodc.eu/api/v1")
time_range = "2024-01-01/2024-01-30"

search = pc_client.search(
    collections=["SENTINEL1_SIG0_20M"],
    datetime=time_range,
    query={"Equi7_TileID": {"eq": "EU020M_E051N018T3"}}
)

items_eodc = search.item_collection()
items_eodc

Sort them by ascending date, and group them if they have the same parent file:

In [37]:
item_list = list(items_eodc)[::-1]
grouped_items = bzs.group_dates(item_list)

In [38]:
grouped_items[0]

[<Item id=SIG0_20240101T165956_A044_E051N018T3_EU020M_V1M1R2_S1AIWGRDH>,
 <Item id=SIG0_20240101T170021_A044_E051N018T3_EU020M_V1M1R2_S1AIWGRDH>,
 <Item id=SIG0_20240101T170046_A044_E051N018T3_EU020M_V1M1R2_S1AIWGRDH>]

In [57]:
a = bzs.load_data(item_list[0], ["VH", "VV"])
#b = bzs.load_data(item_list[1], ["VH", "VV"])
#c = bzs.load_data(item_list[2], ["VH", "VV"])

In [58]:
a["x"] = a.x-10
a["y"] = a.y+10

In [59]:
a

<xarray.Dataset> Size: 900MB
Dimensions:      (x: 15000, y: 15000)
Coordinates:
    time         datetime64[ns] 8B 2024-01-01T16:59:56
    band         int64 8B 1
  * x            (x) float64 120kB 5.1e+06 5.1e+06 5.1e+06 ... 5.4e+06 5.4e+06
  * y            (y) float64 120kB 2.1e+06 2.1e+06 2.1e+06 ... 1.8e+06 1.8e+06
    spatial_ref  int64 8B 0
Data variables:
    VH           (y, x) int16 450MB -9999 -9999 -9999 ... -9999 -9999 -9999
    VV           (y, x) int16 450MB -9999 -9999 -9999 ... -9999 -9999 -9999
Attributes: (12/32)
    abs_orbit_number:       51916
    border_noise_removed:   True
    creation_date:          2024-01-01 19:15:54
    creator:                gfm_ops
    dem_source:             /store_frei/eodc_gfm/datapool/copdem/CopDEM30m_gl...
    egm_applied:            False
    ...                     ...
    wrapper_git_tag:        v1.1.0
    wrapper_name:           gfm-sigma-nrt
    AREA_OR_POINT:          Area
    _FillValue:             -9999
    scale_factor:           0.1
    add_offset:             0.0

In [60]:
def get_idx(array1, array2):
    min = np.where(array1==array2[0])[0][0]
    max = np.where(array1==array2[-1])[0][0]+1
    return min, max

In [61]:
store = zarr.storage.LocalStore("/eodc/private/openeo_platform/zarr_nacho/s1sig0.zarr")
group = zarr.group(store=store)
x_extent = group["x"][:]
y_extent = group["y"][:]

In [62]:
x_min, x_max = get_idx(x_extent, a["x"].values)
y_min, y_max = get_idx(y_extent, a["y"].values)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [72]:
y_extent

array([1800000., 1799980., 1799960., ..., 1200060., 1200040., 1200020.])

In [70]:
a["y"].values

array([2100000., 2099980., 2099960., ..., 1800060., 1800040., 1800020.])

In [ ]:
origin = np.datetime64("2014-10-01T00:00:00.000000000").astype("datetime64[s]")
sensing_delta = int((a.time.values.astype("datetime64[s]")-origin).astype(int))

In [ ]:
sensing_delta

In [ ]:
arr = a["VH"].astype(np.int64)
test = np.where(arr!=-9999, sensing_delta, arr)

In [ ]:
test

In [ ]:
a.assign(sensing_date=(("x", "y", "time"), ))

In [12]:
crs = CRS.from_wkt(item_list[0].properties["proj:wkt2"])
res = item_list[0].properties["gsd"]
epsg = crs.to_epsg()
bbox = item_list[0].properties["proj:bbox"]

In [31]:
dataset = stackstac.stack(items_eodc,
                        epsg=epsg,
                        assets=["VH", "VV"],
                        bounds = bbox,
                        resolution=res,
                        fill_value=-9999,
                        rescale=False,
                        snap_bounds=False).squeeze()

dataset.isel(time=0)

<xarray.DataArray 'stackstac-85395bacac446be6428d2aefa4610fff' (band: 2,
                                                                y: 15000,
                                                                x: 15000)> Size: 4GB
dask.array<getitem, shape=(2, 15000, 15000), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/36)
    time                                   datetime64[ns] 8B 2024-01-01T16:59:56
    id                                     <U60 240B 'SIG0_20240101T165956_A0...
  * band                                   (band) <U2 16B 'VH' 'VV'
  * x                                      (x) float64 120kB 5.1e+06 ... 5.4e+06
  * y                                      (y) float64 120kB 2.1e+06 ... 1.8e+06
    scale_factor                           object 8B 10
    ...                                     ...
    proj:shape                             object 8B {15000}
    border_noise_removed                   <U4 16B 'True'
    description                            (band) <U47 376B 'The Sigma Nought...
    title                                  (band) <U15 120B 'VH Polarization'...
    raster:bands                           object 8B {'scale': 10, 'nodata': ...
    epsg                                   int64 8B 27704
Attributes:
    spec:        RasterSpec(epsg=27704, bounds=[5100000, 1800000, 5400000, 21...
    crs:         epsg:27704
    transform:   | 20.00, 0.00, 5100000.00|\n| 0.00,-20.00, 2100000.00|\n| 0....
    resolution:  20

In [ ]:
dataset.drop_vars(set(dataset.coords)-set(["time", "x", "y", "band"])).sel(band="VH").isel(time=1).load()

### Write Data to zarr store

Set paths to zarr store, and to respective group to write data to:

In [ ]:
store = zarr.storage.LocalStore("EO_Data.zarr")
group = zarr.group(store=store)["s1sig0/EU"]

Read data from STAC item:

In [ ]:
data_vh = bzs.load_data(items_eodc[0], "VH").squeeze()
data_vh

Data has 15000x15000 pixel values, most of them are often no data values. To improve writing speed the data is clipped to a rectangular extent containing all data values:

In [ ]:
mask = data_vh!=-9999
ymin, ymax = [np.where(mask)[0].min(), np.where(mask)[0].max()+1]
xmin, xmax = [np.where(mask)[1].min(), np.where(mask)[1].max()+1]

data_vh = data_vh.isel(x=slice(xmin, xmax), y=slice(ymin,ymax))

For compression, the time is set to days/seconds after origin. Once for time coordinate and once for the sensing_date metadata array:

In [ ]:
date_origin = np.datetime64("2014-10-01")
date = data_vh.time.values.astype("datetime64[D]")
date_delta = (date - date_origin).astype("int64")

In [ ]:
datetime_origin = np.datetime64("2014-10-01T00:00:00")
datetime = data_vh.time.values.astype("datetime64[s]")
datetime_delta = (datetime - datetime_origin).astype("int64")

Get the indices of the extent in the zarr store:

In [ ]:
x_min, x_max = bzs.get_idx(x_extent, data_vh["x"].values)
y_min, y_max = bzs.get_idx(y_extent, data_vh["y"].values)

As the written data is always rectangular, but the data values are mostly not, the data gaps in the new data which is written to the store have to be filled with potential data values which already exist to prevent overwriting exisiting data with no data values:

In [ ]:
existing_data_vh = group["VH"][date_delta, y_min:y_max, x_min:x_max]
new_data = np.where(data_vh.values==-9999, existing_data_vh, data_vh.values)

As the sensing date is not available as an array but only as metadata for a whole array, it needs to be written to an array in a shape where actual data exists.

In [ ]:
new_sensing = new_data.astype(np.int64)
new_sensing[new_sensing!=-9999] = int(datetime_delta)

Lastly the data can be written to the respective place in the store:

In [ ]:
group["VH"][date_delta, y_min:y_max, x_min:x_max] = new_data
group["sensing_date"][date_delta, y_min:y_max, x_min:x_max] = new_sensing

All the above processes can be done in a loop with custom functions:

In [ ]:
store = zarr.storage.LocalStore("s1sig0.zarr")
group = zarr.group(store=store)["AT"]
x_extent = group["x"][:]
y_extent = group["y"][:]

In [ ]:
bzs.load_data(item_list[0], "VH")

In [28]:
dataset = dataset.assign_coords(time_days=("time", dataset.time.values.astype("datetime64[D]")))
keep_dims = {"time", "x", "y", "band", "time_days", "sat:relative_orbit"}
drop_dims = set(dataset.coords) - keep_dims
dataset = dataset.drop_vars(drop_dims)

In [29]:
dataset

<xarray.DataArray 'stackstac-85395bacac446be6428d2aefa4610fff' (time: 55,
                                                                band: 2,
                                                                y: 15000,
                                                                x: 15000)> Size: 198GB
dask.array<fetch_raster_window, shape=(55, 2, 15000, 15000), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 440B 2024-01-01T16:59:56 ... 2024-01-29T...
  * band       (band) <U2 16B 'VH' 'VV'
  * x          (x) float64 120kB 5.1e+06 5.1e+06 5.1e+06 ... 5.4e+06 5.4e+06
  * y          (y) float64 120kB 2.1e+06 2.1e+06 2.1e+06 ... 1.8e+06 1.8e+06
    time_days  (time) datetime64[s] 440B 2024-01-01 2024-01-01 ... 2024-01-29
Attributes:
    spec:        RasterSpec(epsg=27704, bounds=[5100000, 1800000, 5400000, 21...
    crs:         epsg:27704
    transform:   | 20.00, 0.00, 5100000.00|\n| 0.00,-20.00, 2100000.00|\n| 0....
    resolution:  20

In [ ]:
blocksize=1000
data_y, data_x = dataset.sizes["y"], dataset.sizes["x"]

for y_start in range(0, data_y, blocksize):
    y_end = min(y_start + blocksize, data_y)
    
    for x_start in range(0, data_x, blocksize):
        x_end = min(x_start + blocksize, data_x)
        
        print("start")
        data = dataset.isel(x=slice(x_start, x_end), y=slice(y_start, y_end)).compute()
        print("continues")
        grouped_data = data.groupby("time_days")

        loaded_data = read_data(grouped_data)
        filled_data = fill_data(loaded_data, "2024-01-11", "2024-02-09")

        time_origin = np.datetime64("2014-10-01")
        time_min = (filled_data.time_days.values[0].astype("datetime64[D]") - time_origin).astype("int64")
        time_max = (filled_data.time_days.values[-1].astype("datetime64[D]") - time_origin).astype("int64")+1

        x_min, x_max = bzs.get_idx(x_extent, filled_data["x"].values)
        y_min, y_max = bzs.get_idx(y_extent, filled_data["y"].values)

        data_vh = filled_data.sel(band="VH").values
        group["VH"][time_min:time_max, y_min:y_max, x_min:x_max] = data_vh

        data_vv = filled_data.sel(band="VH").values
        group["VV"][time_min:time_max, y_min:y_max, x_min:x_max] = data_vv

        sensing_date = filled_data.sel(band="datetime").values
        group["sensing_date"][time_min:time_max, y_min:y_max, x_min:x_max] = sensing_date

        print(x_start, y_start)

In [ ]:
for item in tqdm(item_list, leave=True):

    x_extent = group["x"]

    dataset = bzs.load_data(item, ["VH", "VV"])

    dataset_clipped = bzs.clip_data(dataset, multiple_vars=True)
    aon = dataset_clipped.attrs["abs_orbit_number"]
    ron = dataset_clipped.attrs["rel_orbit_number"]
    dataset = None

    time_origin = np.datetime64("2014-10-01")
    times = dataset_clipped.time.values.astype("datetime64[D]")
    time_delta = (times - time_origin).astype("int64")

    sensing_origin = np.datetime64("2014-10-01T00:00:00")
    sensing = dataset_clipped.time.values.astype("datetime64[s]")
    sensing_delta = (sensing - sensing_origin).astype("int64")

    x_min, x_max = bzs.get_idx(x_extent, dataset_clipped["x"].values)
    y_min, y_max = bzs.get_idx(y_extent, dataset_clipped["y"].values)

    data_vh = dataset_clipped["VH"].values
    existing_data_vh = group["VH"][time_delta, y_min:y_max, x_min:x_max]
    np.copyto(existing_data_vh, data_vh, where=(existing_data_vh==-9999))
    group["VH"][time_delta, y_min:y_max, x_min:x_max] = existing_data_vh
    data_vh = None

    data_vv = dataset_clipped["VV"].values
    existing_data_vv = group["VV"][time_delta, y_min:y_max, x_min:x_max]
    np.copyto(existing_data_vv, data_vv, where=(existing_data_vv==-9999))
    group["VV"][time_delta, y_min:y_max, x_min:x_max] = existing_data_vv
    data_vv = None
    existing_data_vv = None

    new_aon = existing_data_vh.astype(np.int32)
    new_aon[new_aon!=-9999] = aon
    group["absolute_orbit_number"][time_delta, y_min:y_max, x_min:x_max] = new_aon
    new_aon = None

    new_ron = existing_data_vh
    new_ron[new_ron!=-9999] = ron
    group["relative_orbit_number"][time_delta, y_min:y_max, x_min:x_max] = new_ron
    new_ron = None

    new_sensing = existing_data_vh.astype(np.int64)
    existing_data_vh = None
    new_sensing[new_sensing!=-9999] = int(sensing_delta)
    group["sensing_date"][time_delta, y_min:y_max, x_min:x_max][mask] = new_sensing
    new_sensing = None

#### Inspecting the store

In [ ]:
ds = xr.open_zarr("s1sig0.zarr", group="AT", consolidated=True, chunks={})#, decode_times=False)

In [ ]:
filtered = ds.sel(time=slice("2024-01-03T00:00:00.000000000","2024-01-03T00:00:00.000000000") , x=slice(4800010, 4801990), y=slice(1799990, 1798010))
time = datetime.now(timezone.utc)
filtered.load()
print(datetime.now(timezone.utc)-time)

In [ ]:
filtered.load()

In [ ]:
filtered